In [ ]:
pip install openpyxl

In [ ]:
pip install pandas

In [ ]:
pip install bs4

In [ ]:
pip install requests

In [43]:
import pandas as pd
import bs4
import requests
import openpyxl

item = 1
brand_list = []
name_list = []
attribute_list = []
price_list = []

while item <= 63:
  data = requests.get('https://www.bnn.in.th/th/p/smartphone-and-accessories?page='+ str(item) + '&in_stock=false&sort_by=relevance')
  soup = bs4.BeautifulSoup(data.text)

  brand = soup.find_all('div',{'class':'product-label-brand'})
  name = soup.find_all('div',{'class':'product-name'})
  short_attribute = soup.find_all('div',{'class':'product-short-attribute'})
  price = soup.find_all('div',{'class':'product-price'})

  for i in brand:
    brand_list.append(i.text.replace("\n", ""))

  for i in name:
    name_list.append(i.text.replace("\n", ""))

  for i in short_attribute:
    attribute_list.append(i.text.replace("\n", ""))

  for i in price:
    price_list.append(i.text.replace("\n", ""))

  # print('Complete item number: ',item)
  item += 1

table = pd.DataFrame([brand_list, name_list, attribute_list,price_list]).transpose()
table.columns = ['Product Brand', 'Product Name', 'Product Attribute', 'Product Price']
table.set_index('Product Brand')
table.to_excel('All phones and accessories.xlsx',engine='openpyxl') #file is created on google colab cloud

In [ ]:
pip install azure-storage-blob

In [ ]:
pip install yahoo-fin

In [ ]:
pip install xlrd

In [ ]:
from azure.storage.blob import BlobServiceClient
from yahoo_fin import stock_info as si
import json
import openpyxl

#ไปที่ storage account แล้วเลือก access key เพื่อ copy key , account และ conection string ใส่ในช่องว่างแต่ละตัวแปร

storage_account_key = "B0xfYqa1wtzBMr9/g3LOkDznPqxY71USEX+9s7vTlFuFvHIDrBy4voMZmclarrR5uAby1svhsnD5+AStbAV5Ow=="
storage_account_name = "bgthanawat"
connection_string = "DefaultEndpointsProtocol=https;AccountName=bgthanawat;AccountKey=B0xfYqa1wtzBMr9/g3LOkDznPqxY71USEX+9s7vTlFuFvHIDrBy4voMZmclarrR5uAby1svhsnD5+AStbAV5Ow==;EndpointSuffix=core.windows.net"
container_name = "blobinsertdata"

#upload file to blobstorage

def uploadToBlobStorage(file_path,file_name):
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)
  blob_client = blob_service_client.get_blob_client(container=container_name,blob=file_name)
  try:
    if blob_client.exists():
        # Delete blob
        blob_client.delete_blob()
        print("Blob deleted successfully!")
    else:
        print("Blob does not exist!")
    with open(file_path,"rb") as data:
      blob_client.upload_blob(data)
    print(f"Uploaded {file_name}.successfully!")
  except Exception as e:
    print("Failed to upload files to container. Error:" + str(e))

uploadToBlobStorage('/content/All phones and accessories.xlsx','All phones and accessories.xlsx') #ไฟล์ที่อ่านจาก google drive เพื่อส่งไปยังไฟล์ใน container

In [ ]:
import os
from multiprocessing.pool import ThreadPool
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient

# IMPORTANT: Replace connection string with your storage account connection string
# Usually starts with DefaultEndpointsProtocol=https;...
MY_CONNECTION_STRING =   "DefaultEndpointsProtocol=https;AccountName=bgthanawat;AccountKey=B0xfYqa1wtzBMr9/g3LOkDznPqxY71USEX+9s7vTlFuFvHIDrBy4voMZmclarrR5uAby1svhsnD5+AStbAV5Ow==;EndpointSuffix=core.windows.net"

# Replace with blob container name
MY_BLOB_CONTAINER = "blobinsertdata"

# Replace with the local folder where you want downloaded files to be stored
LOCAL_BLOB_PATH = "/content/download_data"

class AzureBlobFileDownloader:
  def __init__(self):
    print("Intializing AzureBlobFileDownloader")

    # Initialize the connection to Azure storage account
    self.blob_service_client =  BlobServiceClient.from_connection_string(MY_CONNECTION_STRING)
    self.my_container = self.blob_service_client.get_container_client(MY_BLOB_CONTAINER)

  def download_all_blobs_in_container(self):
    # get a list of blobs
    my_blobs = self.my_container.list_blobs()
    result = self.run(my_blobs)
    print(result)

  def run(self,blobs):
    # Download 10 files at a time!
    with ThreadPool(processes=int(10)) as pool:
     return pool.map(self.save_blob_locally, blobs)

  def save_blob_locally(self,blob):
    file_name = blob.name
    print(file_name)
    bytes = self.my_container.get_blob_client(blob).download_blob().readall()

    # Get full path to the file
    download_file_path = os.path.join(LOCAL_BLOB_PATH, file_name)
    # for nested blobs, create local path as well!
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)

    with open(download_file_path, "wb") as file:
      file.write(bytes)
    return file_name

# Initialize class and upload files
azure_blob_file_downloader = AzureBlobFileDownloader()
azure_blob_file_downloader.download_all_blobs_in_container()

In [ ]:
import pandas as pd
df = pd.read_excel('/content/download_data/All phones and accessories.xlsx')

print(df)